In [107]:
# 1. Загрузка библиотек

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from random import randint
from tqdm import tqdm

In [ ]:
#2. Определение функций сбора, фильтрации и форматирования

In [2]:
news_links = []

def get_news_links(start_page, end_page):

    re_links=[]
    for it in tqdm(range(start_page,end_page)):
        temp_url = "https://www.securitylab.ru/news/page1_" + str(it) + ".php"
        temp_page = rq.get(temp_url)
        temp_page.encoding = "UTF-8"
        soup=BeautifulSoup(temp_page.text, features = "html.parser")
        for link in soup.find_all("a"):
            for i in re.findall(r"news/\d+", str(link.get("href"))):
                if len(str(i)) > 1:
                    re_links.append(str(i))
                else:
                    pass
        time.sleep(randint(2,4))    
    for i in re_links:
        news_links.append("https://www.securitylab.ru/"+i+".php")

In [3]:
texts = []
titles = []
subtitles = []
tags = []

def get_all(news_links):
    for i in tqdm(range(len(news_links))):
        temp_tags = []
        temp_url = news_links[i]
        temp_page = rq.get(temp_url)
        temp_soup = BeautifulSoup(temp_page.text, "html.parser")
        titles.append(temp_soup.title.text.strip())
        subtitles.append(temp_soup.p.text.strip())
        for i in temp_soup.find_all("a", class_ = "tag tag-outline-primary"):
            temp_tags.append(i.text.strip())
        temp_tags=list(set(temp_tags))
        tags.append(tuple(temp_tags))
        temp_string=""
        temp_string += temp_soup.find("div", itemprop = "description").text.strip()
        texts.append(temp_string)
        time.sleep(randint(2,4))
    return texts, titles, subtitles, tags

In [4]:
filtered_texts =[]
filtered_tags = []
filtered_titles = []
filtered_subtitles = []
filtered_news_links = []

def filter(texts, titles, subtitles, tags, news_links):
    for i in texts:
        if type(i) == str:
            filtered_texts.append(texts[texts.index(i)])
            filtered_titles.append(titles[texts.index(i)])
            filtered_subtitles.append(subtitles[texts.index(i)])
            filtered_tags.append(tags[texts.index(i)])
            filtered_news_links.append(news_links[texts.index(i)])
    return filtered_texts, filtered_titles, filtered_subtitles, filtered_tags, filtered_news_links

In [5]:
clear_texts=[]

def clear(filtered_texts):
    for i in filtered_texts:
        t=re.sub(r"(\r\n\s*\t)","",i)
        ct=re.sub(r"\s{2,10}|\r+|\n+|\t+"," ",t)
        clear_texts.append(ct)
    return clear_texts

In [6]:
def save_df(name,filtered_titles,filtered_subtitles,filtered_tags,clear_texts):

    data_list = filtered_titles,filtered_subtitles,filtered_tags,clear_texts

    SecurityLab_df = pd.DataFrame(data_list, columns = filtered_news_links).T
    SecurityLab_df = SecurityLab_df.rename(columns={0:"Title", 1:"Subtitle", 2:"Tags", 3:"Text"})
    SecurityLab_df.to_csv(name+'.csv', encoding='utf-8')

In [ ]:
#3. Сбор содержимого, фильтрация и сохранение

In [7]:
get_news_links(1,406)

100%|████████████████████████████████| 1/1 [00:02<00:00,  2.46s/it]


In [8]:
get_all(news_links)

100%|██████████████████████████████| 15/15 [00:47<00:00,  3.15s/it]


(['Оператор сотовой связи в США Google Fi заявил, что у одного из ее основных сетевых провайдеров произошла утечка личных данных клиентов, а некоторые пользователи заявили об атаках с подменой SIM-карты (SIM Swapping, SIM-свопинг).\r\n  \n\r\n  \t Google разослал клиентам Google Fi   уведомления  \r\n  \tоб утечке данных, сообщив им, что в результате инцидента были раскрыты:\r\n  \n\n\n\n\r\n  \tномера телефонов;\nсерийные номера SIM-карт;\nстатус учетной записи (активный или неактивный);\nдата активации учетной записи;\nсведения о тарифном плане мобильной связи.\n\n\r\n  \t Google пояснил, что взломанные системы не содержат конфиденциальных данных, таких как полные имена, email-адреса, данные платежных карт, номера социального страхования (SSN), ИНН (TIN), удостоверения личности (ID), пароли учетных записей или тексты SMS и записи телефонных звонков.\r\n  \n\r\n  \t Google подтвердил факт несанкционированного доступа и заявил, что расследует инцидент и ведёт работу по защите данных. Т

In [9]:
filter(texts, titles, subtitles, tags, news_links)

(['Оператор сотовой связи в США Google Fi заявил, что у одного из ее основных сетевых провайдеров произошла утечка личных данных клиентов, а некоторые пользователи заявили об атаках с подменой SIM-карты (SIM Swapping, SIM-свопинг).\r\n  \n\r\n  \t Google разослал клиентам Google Fi   уведомления  \r\n  \tоб утечке данных, сообщив им, что в результате инцидента были раскрыты:\r\n  \n\n\n\n\r\n  \tномера телефонов;\nсерийные номера SIM-карт;\nстатус учетной записи (активный или неактивный);\nдата активации учетной записи;\nсведения о тарифном плане мобильной связи.\n\n\r\n  \t Google пояснил, что взломанные системы не содержат конфиденциальных данных, таких как полные имена, email-адреса, данные платежных карт, номера социального страхования (SSN), ИНН (TIN), удостоверения личности (ID), пароли учетных записей или тексты SMS и записи телефонных звонков.\r\n  \n\r\n  \t Google подтвердил факт несанкционированного доступа и заявил, что расследует инцидент и ведёт работу по защите данных. Т

In [10]:
clear(filtered_texts)

['Оператор сотовой связи в США Google Fi заявил, что у одного из ее основных сетевых провайдеров произошла утечка личных данных клиентов, а некоторые пользователи заявили об атаках с подменой SIM-карты (SIM Swapping, SIM-свопинг). Google разослал клиентам Google Fi уведомления об утечке данных, сообщив им, что в результате инцидента были раскрыты:номера телефонов; серийные номера SIM-карт; статус учетной записи (активный или неактивный); дата активации учетной записи; сведения о тарифном плане мобильной связи. Google пояснил, что взломанные системы не содержат конфиденциальных данных, таких как полные имена, email-адреса, данные платежных карт, номера социального страхования (SSN), ИНН (TIN), удостоверения личности (ID), пароли учетных записей или тексты SMS и записи телефонных звонков. Google подтвердил факт несанкционированного доступа и заявил, что расследует инцидент и ведёт работу по защите данных. Также компания уточнила, что системы Google не затронуты. Хотя Google не упомянул, 

In [11]:
save_df("SecurityLab(6000)",filtered_titles,filtered_subtitles,filtered_tags,clear_texts)